In [1]:
import polars as pl

from code.utils.utils import get_int_from_
from code.utils.settings import DefaultPath

Ora ho i risultati di JOSIE-top3 in formato csv 'set_id,s1,o1,s2,o2,s3,o3', voglio andare a ottenere un qualcosa del tipo 'l_id,r_id,o' e raggruppare le coppie così ottenute per l'overlap, con intervali tipo [1,10), [10,50),[50,100)... e poi su questi intervalli vedo quali coppie hanno il corrispetivo nei risultati di SLOTH.

Su queste coppie poi si possono fare meglio delle considerazioni sulla similarità data dagli embedding anche, perché è rimosso (?) il bias della Jaccard similarity.

In [2]:
n = 10000

In [3]:
id_table_file = DefaultPath.josie_stuff_path.base + f'josie-test-{n}/id_table.csv'
id_conversion_table = pl.read_csv(id_table_file)
sloth_results = pl.scan_csv(DefaultPath.data_path.wikitables + 'threshold-r5-c2-a50/sloth-results-r5-c2-a50.csv')

josie_ids, sloth_ids = list(id_conversion_table.to_dict(as_series=False).values())

In [4]:
converter = {j: s for j, s in zip(josie_ids, sloth_ids)}

In [5]:
def split_numbers(s):
    nums = get_int_from_(s)
    ids = [nums[0], nums[2], nums[4]]
    ovs = [nums[1], nums[3], nums[5]]

    return [ids, ovs]

josie_results = pl.scan_csv(DefaultPath.josie_stuff_path.base + f'josie-test-{n}/result_k_3.csv')
pairs = pl.LazyFrame(schema=['r_id', 's_id', 'overlap'])

josie_results = josie_results \
    .select(['query_id', 'results']) \
        .with_columns(
            pl.col('results') \
                .map_elements(split_numbers, pl.List(pl.List(pl.Int64)))
            ) \
                .with_columns(pl.col('results').list.to_struct()) \
                    .unnest('results') \
                        .explode('field_0', 'field_1') \
                            .collect()               

# josie_results.columns = ['r_id', 's_id', 'overlap']
# josie_results = josie_results \
#     .drop_nulls() \
#         .select(
#             pl.when(pl.col('r_id') < pl.col('s_id')).then(pl.col('s_id')).otherwise(pl.col('r_id')),
#             pl.when(pl.col('r_id') < pl.col('s_id')).then(pl.col('r_id')).otherwise(pl.col('s_id')),
#             pl.col('overlap')    
#         )
# josie_results = josie_results.unique(subset=['r_id', 's_id']).sort('overlap')

josie_results = josie_results.drop_nulls().map_rows(lambda t: (converter[t[0]], converter[t[1]], t[2]))
josie_results.columns = ['r_id', 's_id', 'overlap']
josie_results

r_id,s_id,overlap
str,str,i64
"""24216428-1""","""9996355-1""",20
"""24216428-1""","""24216428-2""",19
"""24216428-1""","""20010164-2""",11
"""9650855-2""","""9612244-2""",27
"""9650855-2""","""9628519-2""",27
…,…,…
"""22194385-2""","""16475656-2""",2
"""22194385-2""","""22179947-1""",2
"""22194841-2""","""22194359-1""",3


In [9]:
josie_results.lazy().join(sloth_results, on=['r_id', 's_id']).collect().sort('overlap')

r_id,s_id,overlap,jsim,o_a,a%
str,str,i64,f64,i64,f64
"""23797865-1""","""23797865-6""",2,0.724138,64,0.888889
"""23797865-1""","""23797865-7""",2,0.724138,64,0.888889
"""16475648-3""","""16475656-2""",3,1.0,56,1.0
"""23797865-2""","""23797865-3""",3,0.814815,64,0.888889
"""30992319-2""","""30992337-2""",4,1.0,76,1.0
…,…,…,…,…,…
"""16517223-1""","""16559174-1""",765,0.81258,1190,0.449735
"""16559174-1""","""16682980-1""",795,0.842894,1465,0.549925
"""16519520-1""","""16559174-1""",803,0.867637,1345,0.51063
